<p  style="color: #d34813 ; font-size:30px; font-weight:bold; text-align:center">Book Scraping Notebook</p>


## Import Libraries 

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

* [A. Scrap First Page](#A.-Scrap-First-Page)
    * [1. Get HTML](#1.-Get-Html)
    * [2. Scraping](#2.-Scraping)
* [B. Scrap All Website](#B.-Scrap-All-Website)

## A. Scrap First Page

### 1. Get Html

In [2]:
req = requests.get("https://books.toscrape.com/catalogue/page-1.html")
req

<Response [200]>

**Response 200 requset send and respone get sucessful**

In [3]:
req = req.text # to get text of response (HTML)
page = BeautifulSoup(req,"lxml") # Parsing text to HTML 
print(page.prettify()) # print html code in understandable way

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:30" name="created"/>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="../static/oscar/favicon.ico" rel="shortcut icon"/>
  <link href="../static/oscar/css/styles.css" rel="stylesheet" typ

### 2. Scraping

<p style="color:red; font-weight:bold;">Want to scrap: </p>
<ul>
    <li>Name of book</li>
    <li>Description</li>
    <li>Rate</li>
    <li>Price (incl.tax, excl.tax)</li>
    <li>Tax</li>
    <li>In stock or out</li>
    <li>Number in stock</li>
    <li>Code</li>
</ul>

In [4]:
# Get all containers of books
pages_of_books = page.find_all("li",class_="col-xs-6 col-sm-4 col-md-3 col-lg-3")

# DataFrame to save my csv file
data = pd.DataFrame(columns=["Name","Description","rate","Price incl","Price excl","Tax","Stock","Number","Code"])

for link in pages_of_books:
    
    # Get link for each book page to get more information
    href = link.a.attrs["href"].strip()
    req = requests.get(f"https://books.toscrape.com/catalogue/{href}").text
    page = BeautifulSoup(req,"lxml")
    
    name_of_book = page.h1.text
    description = page.find("p",class_=None).text
    rate = page.find("p",class_="star-rating")["class"][1]
    
    # Search in the rows of table to find price incl.tax, price excl.tax, code, and tax
    trs = page.table.find_all("tr")
    for tr in trs:
        if(tr.th.text == "Price (incl. tax)"):
            price_incl = tr.td.text[1:]
        elif(tr.th.text == "Price (excl. tax)"):
            price_excl=tr.td.text[1:]
        elif(tr.th.text == "UPC"):
            code = tr.td.text
        elif(tr.th.text == "Tax"):
            tax = tr.td.text[1:]
        else:
            continue
            
    # Get stock & number in stock       
    st_av = page.find("p",class_="instock availability").text
    available = st_av.split()[2][1:]
    stock = " ".join(st_av.split()[:2])
    
    # Add the row to dataframe
    data.loc[len(data)] = [name_of_book,description,f"{rate.lower()} stars",price_incl,price_excl,tax,stock,available,code]
    
display(data)

,Name,Description,rate,Price incl,Price excl,Tax,Stock,Number,Code
0,A Light in the Attic,It's hard to imagine a world without A Light i...,three stars,£51.77,£51.77,£0.00,In stock,22,a897fe39b1053632
1,Tipping the Velvet,"""Erotic and absorbing...Written with starling ...",one stars,£53.74,£53.74,£0.00,In stock,20,90fa61229261140a
2,Soumission,"Dans une France assez proche de la nÃ´tre, un ...",one stars,£50.10,£50.10,£0.00,In stock,20,6957f44c3847a760
3,Sharp Objects,"WICKED above her hipbone, GIRL across her hear...",four stars,£47.82,£47.82,£0.00,In stock,20,e00eb4fd7b871a48
4,Sapiens: A Brief History of Humankind,From a renowned historian comes a groundbreaki...,five stars,£54.23,£54.23,£0.00,In stock,20,4165285e1663650f
5,The Requiem Red,Patient Twenty-nine.A monster roams the halls ...,one stars,£22.65,£22.65,£0.00,In stock,19,f77dbf2323deb740
6,The Dirty Little Secrets of Getting Your Dream...,Drawing on his extensive experience evaluating...,four stars,£33.34,£33.34,£0.00,In stock,19,2597b5a345f45e1b
7,The Coming Woman: A Novel Based on the Life of...,"""If you have a heart, if you have a soul, Kare...",three stars,£17.93,£17.93,£0.00,In stock,19,e72a5dfc7e9267b2
8,The Boys in the Boat: Nine Americans and Their...,For readers of Laura Hillenbrand's Seabiscuit ...,four stars,£22.60,£22.60,£0.00,In stock,19,e10e1e165dc8be4a
9,The Black Maria,"Praise for Aracelis Girmay:""[Girmay's] every l...",one stars,£52.15,£52.15,£0.00,In stock,19,1dfe412b8ac00530


## B. Scrap All Website

In [17]:
# Delete old data frame and create new one
del data
data = pd.DataFrame(columns=["Name","Description","rate","Price incl","Price excl","Tax","Stock","Number","Code"])

for i in range(1,51):

    link=f"https://books.toscrape.com/catalogue/page-{i}.html"
    req = requests.get(link)
    page = BeautifulSoup(req.text,"lxml")

    pages_of_books = page.find_all("li",class_="col-xs-6 col-sm-4 col-md-3 col-lg-3")
    
    for link in pages_of_books:
        
        href = link.a["href"].strip()
        req = requests.get(f"https://books.toscrape.com/catalogue/{href}").text
        page = BeautifulSoup(req,"lxml")
        
        
        name_of_book = page.h1.text
        try:
            description = page.find("p",class_=None).text
        except:
             description="No Description"
        rate = page.find("p",class_="star-rating")["class"][1]

        trs = page.table.find_all("tr")
        for tr in trs:
            if tr.th.text == "Price (excl. tax)":
                price_excl = tr.td.text[1:]
            elif tr.th.text == "Price (incl. tax)":
                price_incl = tr.td.text[1:]
            elif tr.th.text == "Tax":
                tax = tr.td.text[1:]
            elif tr.th.text=="UPC":
                code = tr.td.text
            else:
                continue

        st_av = page.find("p","instock availability").text.split()
        stock = " ".join(st_av[:2])
        available = st_av[2][1:]
    
        data.loc[len(data)] = [name_of_book,description,f"{rate.lower()} stars",price_incl,price_excl,tax,stock,available,code]
    print(f"page: {i} done")

page: 1 done
page: 2 done
page: 3 done
page: 4 done
page: 5 done
page: 6 done
page: 7 done
page: 8 done
page: 9 done
page: 10 done
page: 11 done
page: 12 done
page: 13 done
page: 14 done
page: 15 done
page: 16 done
page: 17 done
page: 18 done
page: 19 done
page: 20 done
page: 21 done
page: 22 done
page: 23 done
page: 24 done
page: 25 done
page: 26 done
page: 27 done
page: 28 done
page: 29 done
page: 30 done
page: 31 done
page: 32 done
page: 33 done
page: 34 done
page: 35 done
page: 36 done
page: 37 done
page: 38 done
page: 39 done
page: 40 done
page: 41 done
page: 42 done
page: 43 done
page: 44 done
page: 45 done
page: 46 done
page: 47 done
page: 48 done
page: 49 done
page: 50 done


In [18]:
display(data)

,Name,Description,rate,Price incl,Price excl,Tax,Stock,Number,Code
0,A Light in the Attic,It's hard to imagine a world without A Light i...,three stars,£51.77,£51.77,£0.00,In stock,22,a897fe39b1053632
1,Tipping the Velvet,"""Erotic and absorbing...Written with starling ...",one stars,£53.74,£53.74,£0.00,In stock,20,90fa61229261140a
2,Soumission,"Dans une France assez proche de la nÃ´tre, un ...",one stars,£50.10,£50.10,£0.00,In stock,20,6957f44c3847a760
3,Sharp Objects,"WICKED above her hipbone, GIRL across her hear...",four stars,£47.82,£47.82,£0.00,In stock,20,e00eb4fd7b871a48
4,Sapiens: A Brief History of Humankind,From a renowned historian comes a groundbreaki...,five stars,£54.23,£54.23,£0.00,In stock,20,4165285e1663650f
...,...,...,...,...,...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,No Description,one stars,£55.53,£55.53,£0.00,In stock,1,cd2a2a70dd5d176d
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",High school student Kei Nagai is struck dead i...,four stars,£57.06,£57.06,£0.00,In stock,1,bfd5e1701c862ac3
997,A Spy's Devotion (The Regency Spies of London #1),"In Englandâs Regency era, manners and elegan...",five stars,£16.97,£16.97,£0.00,In stock,1,19fec36a1dfb4c16
998,1st to Die (Women's Murder Club #1),"James Patterson, bestselling author of the Ale...",one stars,£53.98,£53.98,£0.00,In stock,1,f684a82adc49f011


In [19]:
# Saving Data set
data.to_csv("Books.csv",index=False)